# Оптимизация модели данных интернет магазина

## Описание проекта
Интернет-магазину необходимо построить анализ эффективности и прибыльности бизнеса, для чего его аналитикам необходимо отвечать на точечные вопросы о тарифах вендоров, стоимости доставки в разные страны, количестве доставленных заказов за последнюю неделю. Изначально все данные о заказах хранятся в одной таблице и представляют из себя весь лог доставки от момента оформления до выдачи заказа покупателю, что ведёт к накоплению большого количества дублирующейся и несистематизированной справочной информации. При поиске необходимой аналитикам информации в такой таблице нагрузка на хранилище будет не оптимальна, придется усложнять запросы, что может привести к ошибкам.

Для оптимизации нагрузки на хранилище необходимо провести миграцию данных о заказах из большой таблицы в отдельные логические таблицы, и построить на их основе удобное для аналитиков представление витрины данных. 

## Этап 1.  Витрина данных

### 1. Изначальная таблица **shipping**

* _id_ — уникальный идентификатор записи в таблице

* *shipping_id* — уникальный идентификатор доставки

* *sale_id* — уникальный идентификатор продажи

* *order_id* — уникальный идентификатор заказа

* *client_id* — уникальный идентификатор клиента

* *payment_amount* — сумма платежа

* *state_datetime* — время обновления состояния заказа

* *product_id* — уникальный идентификатор товара

* *description* — полное описание товара

* *vendor_id* — уникальный идентификатор вендора. К одному вендору может быть привязано множество *sale_id*

* *name_category* — название категории товара

* *base_country* — страна производитель

* *status* - статус доставки по данному *shipping_id*. Может принимать значения in_progress — доставка в процессе, либо finished — доставка завершена

* *state* — промежуточные точки заказа, которые изменяются в соответствии с обновлением информации о доставке по времени *state_datetime*

* *shipping_plan_datetime* — плановая дата доставки

* *hours_to_plan_shipping* — запланированное количество часов на доставку

* *shipping_transfer_description* — строка со значениями *transfer_type* и *transfer_model*, записанными через ":"
    * *transfer_type* - тип доставки
    * *transfer_model* - модель доставки, то есть способ, которым заказ доставляется до точки  

* *shipping_transfer_rate* — процент стоимости доставки для вендора в зависимости от типа и модели доставки, который взимается интернет-магазином для покрытия расходов

* *shipping_country* — страна доставки, учитывая описание тарифа для каждой страны

* *shipping_country_base_rate* — налог на доставку в страну, который является процентом от стоимости payment_amount

* *vendor_agreement_description*  — строка, в которой содержатся данные *agreement_id*, *agreement_number*, *agreement_rate*, *agreement_commission*, записанные через разделитель «:»   
    * *agreement_id* — идентификатор договора.
    * *agreement_number* — номер договора в бухгалтерии.
    * *agreement_rate* — ставка налога за стоимость доставки товара для вендора.
    * *agreement_commission* — доля в платеже являющаяся доходом компании от сделки.

### 2. Метрики, необходимые аналитикам
 
 * *shipping_id*           — уникальный идентификатор доставки.
 
 * *vendor_id*              — уникальный идентификатор вендора 
 
 * *transfer_type*          — тип доставки, указывающий кто берёт ответственность на себя: интернет-магазин или сам вендор  
 
 * *full_day_at_shipping*   — количество полных дней, в течение которых длилась доставка
 
 * *is_delay*               — статус, показывающий просрочена ли доставка
 
 * *is_shipping_finish*     — статус, показывающий, что доставка завершена
 
 * *delay_day_at_shipping*  — количество дней, на которые была просрочена доставка
 
 * *payment_amount*         — сумма платежа пользователя
 
 * *vat*                    — итоговый налог на доставку
 
 * *profit*                 — итоговый доход компании с доставки

### 3. Расчёт метрик

- Значения для *shipping_id*, *vendor_id*, *transfer_type* и *payment_amount* будут браться из характеристик одноимённой таблицы.

- Кол-во полных дней доставки (*full_day_at_shipping*) будет рассчитываеться как разность между датой фактической выдачи заказа (*shipping_end_fact_datetime*) и датой прниятия заказа(*shipping_start_fact_datetime*) и датой.
       * *shipping_start_fact_datetime* - это значение *state_datetime*, когда state заказа перешёл в состояние "booked"
       * *shipping_end_fact_datetime*   - это значение *state_datetime*, когда state заказа перешел в состояние "received" 

- Статус (*is_delay*), показывающий просрочена ли доставка, будет иметь значение "1" - просрочена, или "0" - не просрочена. Само значение будет определяться из условия *shipping_end_fact_datetime* > *shipping_plan_datetime*.

- Метрика *is_shipping_finish* будет принимать значения "1"(заказ завершён) или "0"(заказ не завершён). Определяется по значению status последней по времени записи для каждого заказа(*status* = "finished").

- Метрика *delay_day_at_shipping* будет расчитываться по похожему принципу с *is_delay*, в случае просрочки будет считаться разность между *shipping_plan_datetime* и *shipping_end_fact_datetime*. В ином случае присваиватся значение "0".

- Расчёт итогового налога на доставку в страну(*vat*) будет рассчитываться от итоговой суммы(*payment_amount*) в виде суммы процента налога на доставку в страну(*shipping_country_base_rate*), процента на стоимость доставки товара для вендора(*agreement_rate*) и процента, взимаемого с вендора интернет-магазином за доставку(*shipping_transfer_rate*).

- Итоговый доход компании с каждой доставки(*profit*) будет находиться из произведения итоговой суммы(*payment_amount*) и процента *agreement_commission* первоначальной таблицы.

## Этап 2. Проведение миграции в нормализированную модель представления данных 

В рамках проекта предлагается сформировать следующую модель данных. Данные о статусе заказа будут вынесены в отдельную таблицу **shipping_status**, а общая информация о заказах будет содержаться в таблице **shipping_info**. Так же будут созданы отдельные справочники для информации о доставки товара в разные страны(**shipping_country_rates**), тарифах доставки различных вендоров(**shipping_agreement**) и о типах доставки(**shipping_transfer**).  


### 1. Справочник **shipping_country_rate**

Справочник стоймости доставки товаров в страны. Строится на основе атрибутов *shipping_country* и *shipping_country_base_rate* изначальной таблицы **shipping**.  В качестве первичного ключа вводится серийный идентификатор каждой строчки "id". Типы данных в текущей и последующих таблиц взяты на основе **shipping**.

* SQL- скрипт создания таблицы:

```
create table if not exists shipping_country_rates(
id                           serial,
shipping_country             text,
shipping_country_base_rate   NUMERIC(14,3),
PRIMARY KEY (id)
);
```

* SQL- скрипт миграции данных:

```
insert into shipping_country_rates(shipping_country, shipping_country_base_rate ) 
select shipping_country, shipping_country_base_rate   
from shipping s 
group by shipping_country, shipping_country_base_rate;
```
Группировка по *shipping_country* и *shipping_country_base_rate* сделана для того, чтобы справочник состоял только из уникальных пар полей. 



### 2. Справочник **shipping_agreement**

Справочник тарифов доставки вендора. Формируется путём распределения массива информации из *vendor_agreement_description* по отдельным столбцам с помощью функции *regexp_split_to_array*.

* SQL- скрипт создания таблицы:

```
create table if not exists shipping_agreement(
agreementid                  bigint,                             -- идентификатор договора
agreement_number             text,                               -- номер договора в бухгалтерии
agreement_rate               NUMERIC(14,3),                      -- ставка налога за стоимость доставки товара для вендора
agreement_commission         NUMERIC(14,3),                      -- комиссия, то есть доля в платеже являющаяся доходом компании от сделки
PRIMARY KEY (agreementid)
);
```

* SQL- скрипт миграции данных:

```
insert into shipping_agreement(agreementid, agreement_number, agreement_rate, agreement_commission) 
select  distinct description[1]::bigint as agreementid, 
		description[2] as agreement_number , 
		description[3]::NUMERIC(14,3) as agreement_rate, 
		description[4]::NUMERIC(14,3) as agreement_commission   
from 
	(select regexp_split_to_array(vendor_agreement_description, ':+') AS description from shipping s) as temporary_table;
```


### 3. Справочник **shipping_transfer**

Cправочник о типах доставки. Строится путем извлечения информации из атрибута *shipping_transfer_description*. В качестве первичного ключа вводится серийный идентификатор каждой строчки "id".

* SQL- скрипт создания таблицы:

```
create table if not exists shipping_transfer(
id                            serial,
transfer_type                 text,                             
transfer_model                text,                             -- способ доставки
shipping_transfer_rate        numeric(14, 3),                   -- процент стоимости доставки для вендора в зависимости от типа и модели доставки, который взимается интернет-магазином для покрытия расходов
PRIMARY KEY (id)
);
```

* SQL- скрипт миграции данных:

```
insert into shipping_transfer(transfer_type, transfer_model, shipping_transfer_rate) 
select distinct des[1] as transfer_type, 
		        des[2] as transfer_model ,
		        shipping_transfer_rate
from (select regexp_split_to_array(shipping_transfer_description, ':+') AS des, shipping_transfer_rate from shipping s) as temporary_table
group by des[1], des[2], shipping_transfer_rate;
```
 
### 4. Таблица **shipping_info**

Справочник комиссий по странам с уникальными доставками. Создаётся путем присоединения таблицы **shipping** с уже созданными выше справочниками. В качестве внешних ключей выбраны *transfer_type_id* (с таблицей **shipping_transfer**), *shipping_country_id*( с таблицей **hipping_country_rates**) и *agreementid* (с таблицей **shipping_agreement**).

* SQL- скрипт создания таблицы:

```
create table if not exists shipping_info(
shippingid                    bigint,
vendorid                      bigint,                             
payment_amount                numeric(14,3),                             
shipping_plan_datetime        timestamp, 
transfer_type_id              bigint, 
shipping_country_id           bigint,
agreementid                   bigint,
PRIMARY KEY (shippingid),
FOREIGN KEY (transfer_type_id ) REFERENCES shipping_transfer (id) on update cascade,
FOREIGN KEY (shipping_country_id) REFERENCES shipping_country_rates (id) on update cascade,
FOREIGN KEY (agreementid) REFERENCES shipping_agreement (agreementid) on update cascade);

```

* SQL- скрипт миграции данных:

```
insert into shipping_info(shippingid, vendorid, payment_amount, shipping_plan_datetime, transfer_type_id, shipping_country_id, agreementid  ) 
select distinct s.shippingid, s.vendorid, s.payment_amount, s.shipping_plan_datetime, 
				st.id as transfer_type_id, 
				scr.id as shipping_country_id, 
				(regexp_split_to_array(s.vendor_agreement_description, ':+'))[1]::bigint as agreementid  
from shipping s 
join shipping_transfer st 
	on (regexp_split_to_array(s.shipping_transfer_description, ':+'))[1] = st.transfer_type  and (regexp_split_to_array(s.shipping_transfer_description, ':+'))[2] = st.transfer_model
join shipping_country_rates scr 
	on s.shipping_country = scr.shipping_country;
```


### 5. Таблица **shipping_status**

Таблица последних по времени статусов доставки, включающая информацию по фактическому времени доставки *shipping_end_fact_datetime* и фактическому времени приёма заказа *shipping_start_fact_datetime*. Если заказ ещё находится в процессе доставки, в *shipping_end_fact_datetime* будет проставляться значение *null*.

* SQL- скрипт создания таблицы:

```
create table public.shipping_status(
shippingid                             bigint,
status                                 text,
state                                  text,
shipping_start_fact_datetime           timestamp,
shipping_end_fact_datetime             timestamp null,
PRIMARY KEY (shippingid)
);
```

* SQL- скрипт миграции данных:

```
insert into shipping_status(shippingid , status, state, shipping_start_fact_datetime, shipping_end_fact_datetime) 
with tmt as (select shippingid, status, state, row_number() over (partition by shippingid order by state_datetime DESC) as num, last_value(state_datetime) over (partition by shippingid) as shipping_start_fact_datetime
	     from shipping s)
select  tmt.shippingid, tmt.status, tmt.state, tmt.shipping_start_fact_datetime, tmt1.time_rec as shipping_end_fact_datetime  
from tmt
	left join (select shippingid, state_datetime as time_rec from shipping where state = 'recieved' ) as tmt1
        on tmt.shippingid = tmt1.shippingid
where num = 1; 
```

### 6. Витрина **shipping_datamart**

На основе предложенний, высказанных в п 1.3, получен SQL-скрипт создания представления витрины данных:

```
creat or replace view shipping_datamart as 
select si.shippingid, 
	   vendorid,
	   transfer_type,
	   extract (day from (ss.shipping_end_fact_datetime - ss.shipping_start_fact_datetime)) as full_day_at_shipping,
	   case when shipping_end_fact_datetime is not null then case when shipping_end_fact_datetime > shipping_plan_datetime then 1 else 0 end else  NULL end as is_delay,	   
	   case when status = 'finished' then 1 else 0 end as is_shipping_finish,
	   case when shipping_end_fact_datetime > shipping_plan_datetime then extract (day from (shipping_end_fact_datetime - shipping_plan_datetime))  else 0 end as delay_day_at_shipping,
	   payment_amount,
	   payment_amount * (shipping_country_base_rate + agreement_rate + shipping_transfer_rate) as vat,
	   payment_amount * agreement_commission as profit
from shipping_info si 
join shipping_status ss 
	on si.shippingid = ss.shippingid 
join shipping_transfer st 
	on transfer_type_id = st.id
join shipping_country_rates scr 
	on shipping_country_id  = scr.id
join shipping_agreement sa 
	on si.agreementid = sa.agreementid
```

## Выводы

  1. Изучены требуемые для аналитиков метрики, представлены механизмы их получения.
  2. Созданы DDL-скрипты таблиц нормализованной модели данных, прописаны ограничения для атрибутов. 
  3. Составлены SQL-скрипты миграции из ненормализованной таблицы в сущности новой модели данных.
  4. На основе новой модели данных разработан SQL запрос создания представления витрины с необходимыми для аналитиков метриками. 
